In [ ]:
import sys
!{sys.executable} -m pip install edtf

In [ ]:
import csv
import json
import random
import re
from collections import OrderedDict
from edtf import parse_edtf

In [ ]:
inputFile = "../input/sari_abzug-utf-8_23_04-tsv.txt"
output = "./output"

In [ ]:
with open(inputFile, 'r') as f:
    rawData = json.load(f)

In [ ]:
datefields = ['100$d', '260$c', '260$g', '264$c', '533$d', '600$d', '611$d', '700$d']

In [ ]:
monthsGermanFull = r'(Januar)|(Februar)|(März)|(April)|(Mai)|(Juni)|(Juli)|(August)|(September)|(Oktober)|(November)|(Dezember)'
monthsGermanAbr = r'(Jan)|(Feb)|(Mär)|(Apr)|(Mai)|(Jun)|(Jul)|(Aug)|(Sept)|(Okt)|(Nov)|(Dez)'
monthsFrenchFull = r'(janvier)|(février)|(mars)|(avril)|(mai)|(juin)|(juillet)|(aout)|(septembre)|(octobre)|(novembre)|(décembre)'
monthsFrenchAbr = r'(janv)|(févr)|(mars)|(avril)|(avr)|(mai)|(juin)|(juil)|(aout)|(aou)|(sept)|(oct)|(nov)|(déc)'
monthsEnglishFull = r'(January)|(February)|(March)|(April)|(May)|(June)|(July)|(August)|(September)|(October)|(November)|(December)'
monthsEnglishAbr = r'(Jan)|(Feb)|(Mar)|(Apr)|(May)|(Jun)|(Jul)|(Aug)|(Sep)|(Oct)|(Nov)|(Dec)'
numeralRomans = r'(VIII)|(III)|(IV)|(VI)|(VII)|(V)|(IX)|(X)|II|I'
remaining = r'[A-z]'
unknowns = r'XX|xx'

In [ ]:
dateVariantsPerField = {}
for datefield in datefields:
    dateVariants = {}
    for row in rawData['rows']:
        uuid = row['UUID']
        if datefield in row and row[datefield] != None:
            date = row[datefield]
            for d in date.split('|'):
                if d:
                    genericDate = re.sub(r'\d','_', d)
                    genericDate = re.sub(monthsGermanFull, '🌕🇩🇪', genericDate, flags=re.IGNORECASE)
                    genericDate = re.sub(monthsFrenchFull, '🌕🇫🇷', genericDate, flags=re.IGNORECASE)
                    genericDate = re.sub(monthsEnglishFull, '🌕🇬🇧', genericDate, flags=re.IGNORECASE)
                    genericDate = re.sub(monthsGermanAbr, '🌙🇩🇪', genericDate, flags=re.IGNORECASE)
                    genericDate = re.sub(monthsFrenchAbr, '🌙🇫🇷', genericDate, flags=re.IGNORECASE)
                    genericDate = re.sub(monthsEnglishAbr, '🌙🇬🇧', genericDate, flags=re.IGNORECASE)
                    genericDate = re.sub(unknowns, '❓', genericDate)
                    genericDate = re.sub(numeralRomans, '🏛', genericDate)
                    #genericDate = re.sub(remaining, '', genericDate)
                    genericDate = genericDate.strip()
                    entry = { 'uuid': uuid, 'value': d}
                    if genericDate in dateVariants:
                        dateVariants[genericDate]['count'] += 1
                        dateVariants[genericDate]['examples'].append(entry)
                    else:
                        dateVariants[genericDate] = {
                            'count': 1,
                            'examples': [entry]
                        }
    dateVariantsOrdered = OrderedDict(sorted(dateVariants.items(), key=lambda kv: kv[1]['count'], reverse=True))
    dateVariantsPerField[datefield] = dateVariantsOrdered

In [ ]:
for field in datefields:
    with open(output + '/dateVariants-' + field + '.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(('Format', 'Appearances', 'Example'))
        for k, d in enumerate(dateVariantsPerField[field]):
            csvwriter.writerow((d, dateVariantsPerField[field][d]['count'], dateVariantsPerField[field][d]['examples'][0]['value']))


# Interpret Values

In [ ]:
field = '260$c'

In [ ]:
uncertaintyQualifiers = 'ca\.|ca|circa|um|vermutlich um'
r = {
    'singleYearWithQualifier': r'^\[?(?:' + uncertaintyQualifiers + '|A°|A°\.|Anno|a\[nn\]o|gezeichnet nach der Natur)?\s?(____)\??\]?$',
    'yearRangeWithQualifier': r'(?:ca\.)?\s?(?:zwischen)?\s?(____\??)\s?(?:-|und|ud)\s?(____\??)'
}

def singleYearWithQualifier(dateString):
    year = re.search(r'(\d{4}\??)', dateString).group(1)
    uncertain = re.search(r'(' + uncertaintyQualifiers + ')', dateString)
    if uncertain:
        return year + "?"
    else:
        return year

def yearRangeWithQualifier(dateString):
    years = re.findall(r'(\d{4}\??)', dateString)
    uncertain = re.search(r'(ca)', dateString)
    if uncertain:
        for i, year in enumerate(years):
            if not '?' in year:
                years[i] += '?'
    return "/".join(years)

def interpret(dateString, pattern):
    m = re.search(r['singleYearWithQualifier'], pattern)
    if m:
        return singleYearWithQualifier(dateString)
    
    m = re.search(r['yearRangeWithQualifier'])
    if m:
        return yearRangeWithQualifier(dateString)
    
    return None

In [ ]:
matches = {
    'matched': [],
    'notMatched': []
}
for pattern in dateVariantsPerField[field].keys():

    m = re.search(r['yearRangeWithQualifier'], pattern)
    if m:
        matches['matched'].append(pattern)
        example = random.choice(dateVariantsPerField[field][pattern]['examples'])['value']
        print(example, " --> ", yearRangeWithQualifier(example))
    else:
        matches['notMatched'].append(pattern)

print("Matches")
print("\n".join(matches['matched']))
print("====\nNot matched")
print("\n".join(matches['notMatched']))